# Experiment on the encoding and imputing technics for the Nam network, with and without Z ! 

In [4]:
run init.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Done.


**`NAN` Second set of experiments deciding wheether `using indicator variable is useful`**

In [7]:
"""  Parameters of the experiment """
approach = 'xgboost'

range_dataset_name = ['circles', 'moons', 'blobs']
range_ratio_of_missing_values = [0., 0.1, 0.2, 0.3]
range_ratio_missing_per_class = [[0,0.25], [0.1, 0.3]]
range_imbalance_ratio = [0.5, 0.25, 0.1]
range_missingness_pattern = [1, 3, 4, 5]

range_missing_data_handling = ['without', 'encoding', 'imputation'] #TODO THIS IS WHERE WE SHOULD DO IMUTING BASED ON THE  OTHER EXPERIMENT imputing, encoding 
imputation_method = 'multi_dimensional_weighting'
range_use_missing_indicator_variables = [True, False]

In [9]:
df = create_df(folder_names=['experiments_29_june_duke'])
df = df.append(pd.read_csv("/home01/sam/df_all_experiments.csv"))


In [2]:
#df = pd.read_csv("/home01/sam/notes_on_stats/doc/src/data/experiemnt_5.csv")

In [10]:
df['ratio_missing_per_class_0'] = df['ratio_missing_per_class_0'].astype(float).round(2)
df['ratio_missing_per_class_1'] = df['ratio_missing_per_class_1'].astype(float).round(2)
df['ratio_of_missing_values'] = df['ratio_of_missing_values'].astype(float).round(2)
df

,dataset_name,experiment_number,approach,missing_data_handling,imputation_method,use_missing_indicator_variables,num_samples,imbalance_ratio,missingness_pattern,missingness_mechanism,ratio_of_missing_values,missing_X1,missing_X2,missing_first_quarter,ratio_missing_per_class_0,ratio_missing_per_class_1,auc,Accuracy,F1,MCC,Sensitivity,Specificity,Precision,PPV,NPV,FNR,FDR,FOR,resolution,bandwidth,estimation_time_0,estimation_time_1
0,circles,428,DecisionTree,encoding,without,True,5000,0.50,2,MAR,0.0,True,True,True,NaN,NaN,NaN,0.835,0.851,0.690,0.952,0.720,0.770,0.770,0.938,0.048,0.230,0.062,20,0.2,NaN,NaN
1,circles,853,xgboost,imputation,multi_dimensional_weighting,False,5000,0.25,5,MNAR,NaN,True,True,True,0.1,0.24,NaN,0.900,0.817,0.759,0.921,0.893,0.734,0.734,0.973,0.079,0.266,0.027,20,0.2,NaN,NaN
2,circles,273,multi_distributions,without,without,False,5000,0.50,4,MNAR,NaN,True,True,False,0.1,0.30,NaN,0.567,0.582,0.136,0.611,0.524,0.556,0.556,0.580,0.389,0.444,0.420,20,0.2,NaN,NaN
3,circles,543,DecisionTree,imputation,multi_dimensional_weighting,False,5000,0.50,3,MAR,0.0,True,False,True,NaN,NaN,NaN,0.835,0.851,0.690,0.952,0.720,0.770,0.770,0.938,0.048,0.230,0.062,20,0.2,NaN,NaN
4,circles,457,DecisionTree,encoding,without,True,5000,0.25,1,MCAR,0.2,True,False,None,NaN,NaN,NaN,0.732,0.638,0.524,0.944,0.661,0.482,0.482,0.973,0.056,0.518,0.027,20,0.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,blobs,13,xgboost,encoding,without,True,5000,0.50,2,MAR,0.0,True,True,True,NaN,NaN,NaN,0.979,0.979,0.958,0.980,0.978,0.978,0.978,0.980,0.020,0.022,0.020,20,0.2,NaN,NaN
863,blobs,5,xgboost,without,without,True,5000,0.50,4,MNAR,NaN,True,True,False,0.0,0.25,NaN,0.984,0.984,0.968,0.978,0.990,0.990,0.990,0.978,0.022,0.010,0.022,20,0.2,NaN,NaN
864,blobs,14,xgboost,encoding,without,True,5000,0.50,2,MAR,0.1,True,True,True,NaN,NaN,NaN,0.938,0.937,0.876,0.935,0.941,0.939,0.939,0.938,0.065,0.061,0.062,20,0.2,NaN,NaN
865,blobs,22,xgboost,encoding,without,True,5000,0.50,4,MNAR,NaN,True,True,False,0.1,0.30,NaN,0.946,0.947,0.893,0.968,0.924,0.927,0.927,0.967,0.032,0.073,0.033,20,0.2,NaN,NaN


In [14]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0

save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

n = 0
todo=0
for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:
        
        for missing_data_handling in range_missing_data_handling:

            for use_missing_indicator_variables in range_use_missing_indicator_variables:

                for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

                    for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):
                        
                        n+=1
                        


                        if check_experiment_already_done(df, 
                                                          dataset_name=dataset_name, 
                                                          approach=approach,
                                                          imbalance_ratio=imbalance_ratio, 
                                                          missingness_pattern=missingness_pattern, 
                                                          imputation_method=imputation_method,
                                                          missing_data_handling=missing_data_handling, 
                                                          use_missing_indicator_variables=use_missing_indicator_variables,
                                                          ratio_of_missing_values=ratio_of_missing_values, 
                                                          ratio_missing_per_class_0=ratio_missing_per_class[0] if ratio_missing_per_class is not None else None):


                            #print("Experiment already done.")
                            continue
                        if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1) or (missing_data_handling in ['without', 'encoding'] and imputation_method!=None):
                            #print("Not doing this.")
                            n-=1
                            continue
                            
                            
                        todo +=1
                        print("{} - {} - missing_data_handling {} imputation_method: {} use_missing_indicator_variables: {} imbalance_ratio: {} missingness_pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                                                                                    approach,
                                                                                                                                                                                                                    missing_data_handling,
                                                                                                                                                                                                                    imputation_method, 
                                                                                                                                                                                                                    use_missing_indicator_variables,
                                                                                                                                                                                                                    imbalance_ratio,
                                                                                                                                                                                                                    missingness_pattern,
                                                                                                                                                                                                                    ratio_of_missing_values, 
                                                                                                                                                                                                                    ratio_missing_per_class))
                        
                        #while True:
                        #    pass
n

252

In [ ]:
"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0

save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}

for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:
        
        for missing_data_handling in range_missing_data_handling:

            for use_missing_indicator_variables in range_use_missing_indicator_variables:

                for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

                    for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):




                        if check_experiment_already_done(df, 
                                                          approach=approach,
                                                          dataset_name=dataset_name, 
                                                          imbalance_ratio=imbalance_ratio, 
                                                          missingness_pattern=missingness_pattern, 
                                                          imputation_method=imputation_method,
                                                          missing_data_handling=missing_data_handling, 
                                                          use_missing_indicator_variables=use_missing_indicator_variables,
                                                          ratio_of_missing_values=ratio_of_missing_values, 
                                                          ratio_missing_per_class_0=ratio_missing_per_class[0] if ratio_missing_per_class is not None else None):


                            print("Experiment already done.")
                            continue
                        if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1) or (missing_data_handling in ['without', 'encoding'] and imputation_method!=None):
                            #print("Not doing this.")
                            continue
                            


                        print("{} - {} - missing_data_handling {} imputation_method: {} use_missing_indicator_variables: {} imbalance_ratio: {} missingness_pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                                                                                    approach,
                                                                                                                                                                                                                    missing_data_handling,
                                                                                                                                                                                                                    imputation_method, 
                                                                                                                                                                                                                    use_missing_indicator_variables,
                                                                                                                                                                                                                    imbalance_ratio,
                                                                                                                                                                                                                    missingness_pattern,
                                                                                                                                                                                                                    ratio_of_missing_values, 
                                                                                                                                                                                                                    ratio_missing_per_class))

                        try:
                            start_time = time()

                            dataset = DatasetGenerator(dataset_name=dataset_name, 
                                                    num_samples=num_samples, 
                                                    imbalance_ratio=imbalance_ratio, 
                                                    missing_data_handling=missing_data_handling,
                                                    imputation_method=imputation_method,
                                                    verbosity=verbosity)

                            # Creation of the missingness
                            dataset.generate_missing_coordinates(missingness_pattern=missingness_pattern, 
                                                                 ratio_of_missing_values=ratio_of_missing_values, 
                                                                 ratio_missing_per_class=ratio_missing_per_class)

                            dataset.split_test_train()

                            # Create the experiments
                            exp = Experiments(dataset_name, 
                                              dataset=dataset, 
                                              purpose='classification', 
                                              approach = approach, 
                                              use_missing_indicator_variables=use_missing_indicator_variables,
                                              previous_experiment=None, 
                                              save_experiment=save, 
                                              verbosity=verbosity, 
                                              debug=False, 
                                              proportion_train=PROPORTION_TRAIN, 
                                              resolution=RESOLUTION, 
                                              bandwidth=BANDWIDTH,
                                              random_state=RANDOM_STATE)


                            # Estimate the distributions 
                            exp.fit()

                            # Estimate the distributions 
                            exp.predict()

                            end_time = time()

                            hours, rest = divmod(end_time - start_time, 3600)
                            minutes, seconds = divmod(rest, 60)
                            print("Done ({}h {}m and {:.2f}s).\n".format(int(hours), int(minutes), seconds))
                        except:
                            print("/!\. Error!\n")
                            print("{} - {} - missing_data_handling {} imputation_method: {} use_missing_indicator_variables: {} imbalance_ratio: {} missingness_pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                                                                                    approach,
                                                                                                                                                                                                                    missing_data_handling,
                                                                                                                                                                                                                    imputation_method, 
                                                                                                                                                                                                                    use_missing_indicator_variables,
                                                                                                                                                                                                                    imbalance_ratio,
                                                                                                                                                                                                                    missingness_pattern,
                                                                                                                                                                                                                    ratio_of_missing_values, 
                                                                                                                                                                                                                    ratio_missing_per_class))                        
                        

# Create final dataset

In [18]:
df_final = pd.DataFrame(columns = ['dataset_name','experiment_number', 'approach', 'missing_data_handling','imputation_method', 'use_missing_indicator_variables', 
                                'num_samples', 'imbalance_ratio', 'missingness_pattern', 'missingness_mechanism', 
                                'ratio_of_missing_values', 'missing_X1', 'missing_X2', 'missing_first_quarter','ratio_missing_per_class_0', 'ratio_missing_per_class_1','auc',
                                'Accuracy', 'F1', 'MCC', 'Sensitivity', 'Specificity', 'Precision', 'PPV', 'NPV', 'FNR', 'FDR', 'FOR', 
                                'resolution', 'bandwidth', 'estimation_time_0', 'estimation_time_1'])



"""  Fix parameters """
#Dataset default parameters
num_samples = NUM_SAMPLES

# pdf estimation default parameters
resolution = RESOLUTION
bandwidth = BANDWIDTH

# Classification default parameters
proportion_train = PROPORTION_TRAIN

# Verbosity
verbosity=0

save=True

missingness_dictionnary = {1: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           2: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           3: (range_ratio_of_missing_values, len(range_ratio_of_missing_values)*[None]),
                           4: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class),
                           5: (len(range_ratio_missing_per_class)*[None], range_ratio_missing_per_class)}
n=0
for dataset_name in range_dataset_name:
    
    for imbalance_ratio in range_imbalance_ratio:
        
        for missing_data_handling in range_missing_data_handling:

            for use_missing_indicator_variables in range_use_missing_indicator_variables:

                for missingness_pattern, (ratio_of_missing_values_list, ratio_missing_per_class_list) in missingness_dictionnary.items():

                    for ratio_of_missing_values, ratio_missing_per_class in zip(ratio_of_missing_values_list, ratio_missing_per_class_list):
                        
                        n+=1
                        


                        if check_experiment_already_done(df, 
                                                          dataset_name=dataset_name, 
                                                          approach=approach,
                                                          imbalance_ratio=imbalance_ratio, 
                                                          missingness_pattern=missingness_pattern, 
                                                          imputation_method=imputation_method,
                                                          missing_data_handling=missing_data_handling, 
                                                          use_missing_indicator_variables=use_missing_indicator_variables,
                                                          ratio_of_missing_values=ratio_of_missing_values, 
                                                          ratio_missing_per_class_0=ratio_missing_per_class[0] if ratio_missing_per_class is not None else None):


                            #print("Experiment already done.")
       
                            
                            
                            iter_df = check_experiment_already_done(df, 
                                                          dataset_name=dataset_name, 
                                                          approach=approach,
                                                          imbalance_ratio=imbalance_ratio, 
                                                          missingness_pattern=missingness_pattern, 
                                                          imputation_method=imputation_method,
                                                          missing_data_handling=missing_data_handling, 
                                                          use_missing_indicator_variables=use_missing_indicator_variables,
                                                          ratio_of_missing_values=ratio_of_missing_values, 
                                                          ratio_missing_per_class_0=ratio_missing_per_class[0] if ratio_missing_per_class is not None else None, return_df = True)
                            
                            if len(iter_df) > 1:
                                df_final = df_final.append(iter_df.iloc[:1])
                                
                            else:
                                df_final = df_final.append(iter_df.iloc[:1])
                                

                            continue
                        if (dataset_name == 'moons' and missingness_pattern in [3, 5]) or (range_ratio_of_missing_values==0 and missingness_pattern != 1) or (missing_data_handling in ['without', 'encoding'] and imputation_method!=None):
                            n-=1
                            
                            continue
                            
                            
                            
                        print("{} - imputation method: {} missing_data_handling: {} imbalance: {} pattern: {} ratio_missing:{} ratio_per_class: {}".format(dataset_name, 
                                                                                                                                                        missing_data_handling,
                                                                                                                                                          imputation_method, 
                                                                                                                                                          imbalance_ratio,
                                                                                                                                                          missingness_pattern,
                                                                                                                                                          ratio_of_missing_values, 
                                                                                                                                                              ratio_missing_per_class))
                        
                       

In [20]:
assert n == len(df_final)

In [21]:
df_final.to_csv("/home01/sam/notes_on_stats/doc/src/data/experiemnt_5.csv", index=False)